In [26]:
import json
import pprint
import pymongo
from pymongo import MongoClient
client = MongoClient()
client = MongoClient('localhost', 27017)
db = client['articles']
collection = db['aggr_arts_for_word']

In [2]:

# f = open("sorted.json", "a")
# f.write("[")
# l = collection.find().distinct('_id')
# for i in range(0,len(l)):
#     e = collection.find_one({'_id': l[i]})
#     list_of_art = e['articles']
#     newlist = sorted(list_of_art, key=lambda k: k['word_count'], reverse=True) 
#     y = json.dumps({'_id':l[i], 'articles':newlist})
#     f.write(y)
#     f.write(",")
#     if i%1000==0:
#         print(i)
        
# f.write("]")
# f.close()

In [27]:
import nltk
import re
import numpy
import json
import time
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
#nltk.download('stopwords')

In [28]:
def get_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [29]:
import pandas as pd 
col_list = ["id", "content"]
data = pd.read_csv("a123.csv", usecols=col_list) 



['A central Texas man serving a life sentence for a double murder in 1992 is innocent, as are three codefendants no longer in prison, a state judge has found. Retired district judge George Allen ruled Friday that Richard Bryan Kussmaul, 45, should be free. His three codefendants each received   sentences and have already been released.  DNA evidence not available two decades ago shows the four weren’t involved in the fatal shootings of    Leslie Murphy and    Stephen Neighbors at a home near Moody, south of Waco, Allen said in a   opinion.  Allen’s recommendation now goes to the Texas court of criminal appeals for a final decision. The state’s top criminal appeals court had ordered a hearing held last month to look into the men’s claims they were innocent.  “The evidence just seemed overwhelming, with the DNA evidence having excluded these men from any contact whatsoever with these people,” Kussmaul’s lawyer, David Sheppard, told the Waco  . Allen presided over the trial where Kussmaul

In [58]:
start_time = time.time()


collection_sort = db['sorted_articles_for_word']
collection_map = db['word_article_map']


stop_words = set(stopwords.words('english')) 
lemmatizer = WordNetLemmatizer()

text = "the election day"
pre_t = text.lower()
pre_t = re.sub('[^A-Za-z0-9]+', ' ', pre_t)

word_tokens = word_tokenize(pre_t)  
word_tags = nltk.pos_tag(word_tokens)
word_tags_size = len(word_tags)

#print(k, word_tags_size,"\n\n")

#i - tokenized word index with POS
pos = []
no_sw_list = []

for i in range(0, word_tags_size):
    if word_tags[i][0] not in stop_words:
        word = lemmatizer.lemmatize(word_tags[i][0], get_pos(word_tags[i][1]))
        if(collection_sort.find_one({'_id': word})):
            no_sw_list.append(word)

no_sw_list



set_pr = {}
set_all = {}
dict_article_fw_c = {}




for i in range(0,len(no_sw_list)):
    x = collection_sort.find_one({'_id': no_sw_list[i]})
    list_of_art = x['articles']
    dict_article_fw_c[no_sw_list[i]] = {d['article_id']:d['word_count'] for d in list_of_art}


pr_set = {}


sets = dict_article_fw_c[no_sw_list[0]].keys()

d_res = {}
d_check = {}
list_of_res = []

w_s = len(no_sw_list)

# if ONE word
if len(no_sw_list)==1:
    d_res = dict_article_fw_c[no_sw_list[0]]
    list_of_res = list(d_res.keys())[0:5]
    
else:
    
    #get articles that contains all of the keywords
    for i in range(0,len(no_sw_list)):

        sets = sets & dict_article_fw_c[no_sw_list[i]].keys()
        if not sets:
            break

    ls = list(sets)
    list_of_sum = []
    new_dict = {}

    for i in ls:
        sum = 0
        for j in dict_article_fw_c:
            sum = sum+dict_article_fw_c[j][i]
        new_dict[i] = sum
    
    list_of_res = list(dict(sorted(new_dict.items(), key=lambda item: item[1], reverse=True)).keys())
    
    
    if list_of_res:
        
        #article next word count
        dict_art_nw_c = {}
        
        for k in range(0,len(no_sw_list)-1):
        
            for i in range(len(list_of_res)):
                
                word = no_sw_list[k]
                nword = no_sw_list[k+1]
                
                x = collection_map.find_one({'_id':{'word': word, 'article_id':list_of_res[i]}})
#                 print(nword)
#                 print(x["nw_key"])
#                 print("\n")
                #print(list_of_res[i],word, nword, x["nw_key"])
                
                if nword in x["nw_key"]:
        
                    if list_of_res[i] in dict_art_nw_c:
                        dict_art_nw_c[list_of_res[i]] += 1
                        
#                         if(dict_art_nw_c[list_of_res[i]]==(w_s-2)):
#                             d_check[list_of_res[i]] = 1
                        
#                         if(d_check>5):
#                             break
                            
                        #print(list_of_res[i])
                    else:
                        dict_art_nw_c[list_of_res[i]] = 1
                        
        list_of_res = list(dict(sorted(dict_art_nw_c.items(), key=lambda item: item[1], reverse=True)))    
    else:
        pass


#if list_of_res<5:
    
print("Sentence: ", text)
print("\n")
# for i in range(len(list_of_res)):
#     print("Rank: ",i+1)
#     print("Artcile ID: ",list_of_res[i])
#     print("\n\n")
#     if i==4:
#         break
        
        
for i in range(len(list_of_res)):
    print("Rank: ",i+1)
    print("Artcile ID: ",list_of_res[i])
    #print("Content: ", list(data['content'].loc[data['id'] == list_of_res[i]]))
    print("")
    if i==4:
        break
        
print("----------------------- Time: %s seconds -----------------------" % (time.time() - start_time))

Sentence:  the election day


Rank:  1
Artcile ID:  56863

Rank:  2
Artcile ID:  206095

Rank:  3
Artcile ID:  78411

Rank:  4
Artcile ID:  78426

Rank:  5
Artcile ID:  36901

----------------------- Time: 15.225021600723267 seconds -----------------------


In [5]:
# collection_sort = db['sorted_articles_for_word']
# collection_map = db['word_article_map']


# stop_words = set(stopwords.words('english')) 
# lemmatizer = WordNetLemmatizer()

# text = "If the court upholds the judge’s opinion and the men’s claims of actual innocence, they each could be eligible for a state payout of $80, 000 for each year in prison. "
# pre_t = text.lower()
# pre_t = re.sub('[^A-Za-z0-9]+', ' ', pre_t)

# word_tokens = word_tokenize(pre_t)  
# word_tags = nltk.pos_tag(word_tokens)
# word_tags_size = len(word_tags)

# #print(k, word_tags_size,"\n\n")

# #i - tokenized word index with POS
# pos = []
# no_sw_list = []

# for i in range(0, word_tags_size):
#     if word_tags[i][0] not in stop_words:
#         word = lemmatizer.lemmatize(word_tags[i][0], get_pos(word_tags[i][1]))
#         no_sw_list.append(word)

# no_sw_list

# collection_sort = db['sorted_articles_for_word']
# collection_map = db['word_article_map']

# set_pr = {}
# set_all = {}
# dict_article_fw_c = {}


# for i in range(0,len(no_sw_list)):
#     x = collection_sort.find_one({'_id': no_sw_list[i]})
#     #pprint.pprint(x)
#     list_of_art = x['articles']
#     #print(list_of_art)
#     #{d['article_id']:d['word_count'] for d in list_of_art}
#     dict_article_fw_c[no_sw_list[i]] = {d['article_id']:d['word_count'] for d in list_of_art}
    
# #     sett_cur = {d['article_id'] for d in list_of_art}
# #     if set_pr:
# #         set_all = set_all
    
    
# #    collection_map.find_one({'word': no_sw_list[i], 'article_id'})

# pr_set = {}


# sets = dict_article_fw_c[no_sw_list[0]].keys()

# d_res = {}
# list_of_res = []

# if len(no_sw_list)==1:
#     d_res = dict_article_fw_c[no_sw_list[0]]
#     list_of_res = list(d_res.keys())[0:5]
    
# else:
#     for i in range(0,len(no_sw_list)):

#         sets = sets & dict_article_fw_c[no_sw_list[i]].keys()
#         if not sets:
#             break

        
#     ls = list(sets)

#     list_of_sum = []
#     new_dict = {}

#     for i in ls:
#         sum = 0
#         for j in dict_article_fw_c:
#             sum = sum+dict_article_fw_c[j][i]
#         new_dict[i] = sum
    
#     list_of_res = list(dict(sorted(new_dict.items(), key=lambda item: item[1], reverse=True)).keys())

    
    
    
# for i in range(len(list_of_res)):
#     print(i+1,list_of_res[i])
#     if i==4:
#         break

1 151912


In [13]:
collection_map.find_one({'word': "go", 'article_id':17283})

In [20]:
x["nw_key"]

['canada', 'outside', 'shots', 'turkish', 'shia', 'accord', 'kill']

In [53]:
a = {'a':5, '3':5}
len(a)

2